In [5]:
from cognite.client import CogniteClient
import os
import time
import json 
# Instantiate Cognite SDK client:
client = CogniteClient()

In [2]:
## Doc Parser Functions
# OUTPUT DIR 
OUTPUT_DIR = "generated_doc_parser"

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

def start_parsing_job(file_id, page_begin, page_end, space, external_id, use_vision, user_prompt):
    """Start a document parsing job"""
    try:
        payload = {
            "files": [
                {
                    "fileId": file_id,
                    "pageRange": {"begin": page_begin, "end": page_end}
                }
            ],
            "viewConfig": {"space": space, "externalId": external_id},
            "useVision": use_vision,
            "userPrompt": user_prompt
        }
        
        response = client.post(
            f"/api/v1/projects/{client.config.project}/context/documentparser/jobs/start",
            json=payload
        )
        
        if response.status_code in [200, 202]:  # Accept both 200 and 202 status codes
            job_id = response.json()['jobId']
            print(f"✅ Parsing job started successfully! Job ID: {job_id}")
            return job_id
        else:
            print(f"❌ Failed to start parsing job: {response.status_code} - {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error starting parsing job: {str(e)}")
        return None

def get_job_status(job_id):
    """Get the status of a parsing job"""
    try:
        response = client.post(
            f"/api/v1/projects/{client.config.project}/context/documentparser/jobs/byids",
            json={"items": [{"jobId": job_id}]}
        )
        
        if response.status_code in [200, 202]:  # Accept both 200 and 202 status codes
            response_data = response.json()
            
            # The response contains an array of items, we want the first (and only) one
            if response_data.get('items') and len(response_data['items']) > 0:
                job_data = response_data['items'][0]
                status = job_data.get('status', {}).get('job', 'UNKNOWN')
                print(f"📊 Job {job_id} status: {status}")
                return job_data
            else:
                print("❌ No job data returned from API")
                return None
        else:
            print(f"❌ Failed to get job status: {response.status_code} - {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error getting job status: {str(e)}")
        return None

def wait_for_job_completion(job_id, max_wait_time=300, check_interval=10):
    """Wait for a job to complete and return the results"""
    start_time = time.time()
    
    while time.time() - start_time < max_wait_time:
        job_data = get_job_status(job_id)
        if job_data:
            status = job_data.get('status', {}).get('job', 'UNKNOWN')
            
            if status == "Completed":
                print(f"🎉 Job {job_id} completed successfully!")
                results = job_data.get('result', {})
                return results
            elif status in ["FAILED", "Failed"]:
                print(f"💥 Job {job_id} failed!")
                return None
            else:
                print(f"⏳ Job {job_id} still running... (status: {status})")
                time.sleep(check_interval)
        else:
            print(f"❌ Could not check status for job {job_id}")
            break
    
    print(f"⏰ Timeout waiting for job {job_id} to complete")
    return None

def wait_and_save_job_results(job_id=None, max_wait_time=300, check_interval=15):
    """
    Wait for a parsing job to complete and save results to file.
    
    Args:
        job_id: The job ID to wait for. 
        max_wait_time: Maximum time to wait in seconds (default: 300)
        check_interval: How often to check status in seconds (default: 15)
    
    Returns:
        dict: Job results if successful, None otherwise
    """
    # Use provided job_id or try to get from globals
    
    if not job_id:
        print("❌ No job_id found. Please run the job start cell first.")
        return None
    
    print(f"⏳ Waiting for job {job_id} to complete...")
    results = wait_for_job_completion(job_id, max_wait_time=max_wait_time, check_interval=check_interval)
    
    if results:
        print("\n📊 Extraction Results:")
        print(json.dumps(results, indent=2))
        
        # Save results to file 
        filename = f"{OUTPUT_DIR}/extraction_results_{job_id}_{int(time.time())}.json"
        with open(filename, 'w') as f:
            json.dump(results, f, indent=2)
        print(f"\n💾 Results saved to: {filename}")
        return results
    else:
        print("\n❌ Job did not complete successfully or timed out")
        return None

# Create function to call to prevent having to call start_parsing_job() in each cell
def start_job():
    """Assume all parameters are defined in memory"""
    job_id = start_parsing_job(
        file_id=file_id,
        page_begin=page_begin, 
        page_end=page_end,
        space=space,
        external_id=external_id,
        use_vision=use_vision,
        user_prompt=user_prompt
    )

    if job_id:
        print(f"\n🚀 Job started! Job ID: {job_id}")
        print("Use the next cell to check status or wait for completion.")
    
    return job_id


In [26]:
# File parameters
file_id = 4036240432700431 
# 52TT427-0002.PDF
# 
page_begin = 2
page_end = 2

# Data model parameters  
space = "pbf_extractor_sp"           
external_id = "InstrumentEntry" 

# Parsing parameters
use_vision = True
user_prompt = """
- For the tag_number_table_information, return information in JSON format.
- Your goal is to extract the list of records from the main table in the document.
- Return the data as a JSON array, where each object in the array corresponds to a single row in the table.
- Ensure each record contains only ONE row of information. 
- For each row, create a JSON object by extracting the data from the following columns:
    * Rev
    * Tag Number
    * Range
    * Type Input
    * P&ID
    * Equip
    * Meas Incr Output
    * Service Description
- Do not return ```json``` or wrap the output in triple backticks.
"""


print("📝 Configuration:")
print(f"File ID: {file_id}")
print(f"Pages: {page_begin}-{page_end}")
print(f"View: {space}/{external_id}")
print(f"Use Vision: {use_vision}")
print(f"User Prompt: {user_prompt if user_prompt else 'None'}")

📝 Configuration:
File ID: 4036240432700431
Pages: 2-2
View: pbf_extractor_sp/InstrumentEntry
Use Vision: True
User Prompt: 
- For the tag_number_table_information, return information in JSON format.
- Your goal is to extract the list of records from the main table in the document.
- Return the data as a JSON array, where each object in the array corresponds to a single row in the table.
- Ensure each record contains only ONE row of information. 
- For each row, create a JSON object by extracting the data from the following columns:
    * Rev
    * Tag Number
    * Range
    * Type Input
    * P&ID
    * Equip
    * Meas Incr Output
    * Service Description
- Do not return ```json``` or wrap the output in triple backticks.



In [27]:
job_id = start_job()

✅ Parsing job started successfully! Job ID: 4655887105002291

🚀 Job started! Job ID: 4655887105002291
Use the next cell to check status or wait for completion.


In [29]:

results = wait_and_save_job_results(job_id=job_id)

⏳ Waiting for job 4655887105002291 to complete...
📊 Job 4655887105002291 status: Completed
🎉 Job 4655887105002291 completed successfully!

📊 Extraction Results:
{
  "data": {},
  "rawResponses": {
    "tag_number_table_information": {
      "fileId": 4036240432700431,
      "otherAnswers": [],
      "pageNum": 2,
      "propertyId": "tag_number_table_information",
      "selectedAnswerIndex": -1,
      "spatialData": null,
      "value": "[{'Rev': '1', 'Tag Number': '52TT008', 'Range': '0 - 350 \u00baF', 'Type Input': 'T/C - K(CA)', 'P&ID': '10091500-52A-001', 'Equip': 'K2301', 'Meas Incr Output': 'INC.', 'Service Description': 'K2301 MAKE-UP COMPRESSOR 1ST STAGE (CYLINDER #2)'}, {'Rev': '1', 'Tag Number': '52TT009', 'Range': '0 - 350 \u00baF', 'Type Input': 'TIC - K(CA)', 'P&ID': '10091500-52A-001', 'Equip': 'K2301', 'Meas Incr Output': 'INC.', 'Service Description': 'K2301 MAKE-UP COMPRESSOR 1ST STAGE DISCHARGE (CYLINDER #4)'}, {'Rev': '0', 'Tag Number': '52TT023', 'Range': '0 - 280 

In [36]:
import ast 

data_string = results['rawResponses']['tag_number_table_information']['value']

In [37]:
list_of_dicts = ast.literal_eval(data_string)

In [38]:
list_of_dicts

[{'Rev': '1',
  'Tag Number': '52TT008',
  'Range': '0 - 350 ºF',
  'Type Input': 'T/C - K(CA)',
  'P&ID': '10091500-52A-001',
  'Equip': 'K2301',
  'Meas Incr Output': 'INC.',
  'Service Description': 'K2301 MAKE-UP COMPRESSOR 1ST STAGE (CYLINDER #2)'},
 {'Rev': '1',
  'Tag Number': '52TT009',
  'Range': '0 - 350 ºF',
  'Type Input': 'TIC - K(CA)',
  'P&ID': '10091500-52A-001',
  'Equip': 'K2301',
  'Meas Incr Output': 'INC.',
  'Service Description': 'K2301 MAKE-UP COMPRESSOR 1ST STAGE DISCHARGE (CYLINDER #4)'},
 {'Rev': '0',
  'Tag Number': '52TT023',
  'Range': '0 - 280 ºF',
  'Type Input': 'TIC - K(CA)',
  'P&ID': '10091500-52A-003',
  'Equip': 'C2345',
  'Meas Incr Output': 'INC.',
  'Service Description': 'K2301 STG 2 DISCH DAMP'},
 {'Rev': '0',
  'Tag Number': '52TT027',
  'Range': '0 - 340 ºF',
  'Type Input': 'TIC - K(CA)',
  'P&ID': '10091500-52A-004',
  'Equip': 'C2338',
  'Meas Incr Output': 'INC.',
  'Service Description': 'K2302 MAKE UP COMP. DISCHARGE'},
 {'Rev': '0',
 

In [41]:
import pandas as pd
df = pd.DataFrame(list_of_dicts)

In [43]:
df.head(10)

Rev Tag Number       Range   Type Input              P&ID  Equip  \
0   1    52TT008  0 - 350 ºF  T/C - K(CA)  10091500-52A-001  K2301   
1   1    52TT009  0 - 350 ºF  TIC - K(CA)  10091500-52A-001  K2301   
2   0    52TT023  0 - 280 ºF  TIC - K(CA)  10091500-52A-003  C2345   
3   0    52TT027  0 - 340 ºF  TIC - K(CA)  10091500-52A-004  C2338   
4   0    52TT037  0 - 340 ºF  TIC - K(CA)  10091500-52A-005  C2342   
5   0    52TT043  0 - 280 ºF  TIC - K(CA)  10091500-52A-006  C2346   
6   0    52TT077  0 - 180 ºF  TIC - K(CA)    15291005-A-108  K2308   
7   0    52TT078  0 - 180 ºF  TIC - K(CA)    15291005-A-108  K2308   
8   0    52TT079  0 - 180 ºF  TIC - K(CA)    15291005-A-108  K2308   
9   0    52TT090  0 - 160 ºF  T/C - K(CA)    15291005-A-108  D2360   

  Meas Incr Output                                Service Description  
0             INC.   K2301 MAKE-UP COMPRESSOR 1ST STAGE (CYLINDER #2)  
1             INC.  K2301 MAKE-UP COMPRESSOR 1ST STAGE DISCHARGE (...  
2             INC.                             K2301 STG 2 DISCH DAMP  
3             INC.                      K2302 MAKE UP COMP. DISCHARGE  
4             INC.        K2302 MAKE UP COMPRESSOR 2ND STAGE DISHARGE  
5             INC.  K2302 MAKE UP COMPRESSOR 3RD STAGE DISCHARGE D...  
6             INC.                      K2308 1ST STAGE COOLING WATER  
7             INC.                      K2308 2ND STAGE COOLING WATER  
8             INC.                      K2308 3RD STAGE COOLING WATER  
9             INC.                                K2308 COOLING WATER